In [1]:
import pickle
import os
import pandas as pd
import numpy as np
np.random.seed(42)
from sklearn import utils
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.neighbors import KNeighborsClassifier as knc
from sklearn.model_selection import train_test_split as splt
from sklearn.metrics import matthews_corrcoef as mcc 
from sklearn.metrics import accuracy_score as acc

In [2]:
dfx = pd.read_csv("PLIC-milano-processed.csv", sep=';')
dfy = pd.read_csv("MILANO_wSCORE.csv")

/home/webvalley/anaconda3/envs/score/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (14,29,36,44,51,94,104,105,112,113,120,127,130,149,158,168,237,239,240,241,248,249,256,273,298) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Thresholds are 0 - 0.01, 0.01 - 0.02, 0.02 - 0.05, 0.05 - 1
y = dfy["SCORE"].values/100
y_cat = np.zeros((y.shape[0]))
for i, val in enumerate(y):
    if(val < 0.01):
        y_cat[i] = 1
    elif(val < 0.02):
        y_cat[i] = 2
    elif(val < 0.05):
        y_cat[i] = 3
    else:
        y_cat[i] = 4

In [4]:
dfx = dfx.select_dtypes(exclude=['object'])
dfx = dfx.drop(labels = ['smoking', 'smoking recod', 'glucose', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1) #
dfx = dfx.dropna(how='any', axis=1)
X = np.around(dfx.values).astype(int)

In [5]:
X_tr, X_ts, y_tr, y_ts = splt(X, y_cat, test_size=0.25, random_state=42)
print(utils.multiclass.type_of_target(X_tr))
print(utils.multiclass.type_of_target(y_tr))

multiclass-multioutput
multiclass


In [6]:
clf = rfc(n_estimators = 1000, max_depth = 100, random_state = 42)
clf.fit(X_tr, y_tr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=100, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [7]:
y_ts_pred = clf.predict(X_ts)


In [8]:
print(mcc(y_ts, y_ts_pred))

0.6343398282454351


In [9]:
print(acc(y_ts, y_ts_pred))

0.7301038062283737
